# Scraping News VK Json

# News

В этом примере, мы скрейпим новости с сайта Медуза. На выходе записываем название статьи, ее тексти и время когда она была опубликована. 

Делаем все как обычно: requests, Beatiful Soup

In [75]:
import requests
import bs4
import pandas as pd


url = ("https://meduza.io")
page = requests.get(url)
releases = page.text

soup = bs4.BeautifulSoup(releases, "html.parser")

links = []

Ищем ссылки на новости. Они находятся в тегах "a", а сама ссылки имеет начало "news/". Этим мы и воспользуемся для проверки ссылки. Если это верная ссылка, то запишем ее в массив

In [76]:
for link in soup.findAll ("a"):
    release = link.get ("href")
    if "news/" in str(release):
        if str(release) not in links:
            links.append(release)

In [77]:
links

['/news/2018/11/08/zhurnalista-cnn-lishili-akkreditatsii-v-belom-dome-prezidentskiy-pul-vyrazil-protest-trampu',
 '/news/2018/11/08/v-buryatii-mvd-potrebovalo-ot-izdaniya-predostavit-dannye-chitateley-ostavivshih-negativnye-kommentarii-k-statie-o-glave-respubliki',
 '/news/2018/11/08/ssha-obvinili-iran-v-popytke-spryatat-neftyanye-tankery',
 '/news/2018/11/08/ria-novosti-13-zaklyuchennyh-pogibli-vo-vremya-bunta-v-kolonii-v-tadzhikistane',
 '/news/2018/11/08/v-moskve-zaderzhali-podozrevaemogo-v-podzhoge-hrama',
 '/news/2018/11/08/umer-hudozhnik-oskar-rabin',
 '/news/2018/11/07/on-gluboko-poryadochnyy-chelovek-i-ochen-odarennyy-rezhisser-kirill-kleymenov-v-efire-pervogo-o-kirille-serebrennikove',
 '/news/2018/11/08/vyacheslava-tsepovyaza-pomestili-v-shtrafnoy-izolyator-on-el-krasnuyu-ikru-i-krabov-v-kolonii',
 '/news/2018/11/08/kommersant-neftekompanii-povysili-tseny-na-toplivo-dlya-korporativnyh-klientov',
 '/news/2018/11/07/genprokuror-ssha-dzheff-seshns-podal-v-otstavku',
 '/news/2018

В данных списках будем хранить название статьи, время ее публикации и сам текст

In [95]:
titles = []
dates = []
texts = []

Каждую ссылку открываем и ищем там нужные теги для заголовка статьи ( 'span' класса "NewsMaterialHeader-first"), даты публикации ('div' класса "MaterialMeta MaterialMeta--time") и самого текста статьи ("div" класса "Body"). Но есть загвоздка. Текст в этом теге разбит на параграфы "p". Поэтому нам нужно их собрать вместе. Делаем поиск по всем тегам уже внутри **тега с текстом**, собираем все это в одну строку и уже обойдя все параграфы, записываем в нужный список.

In [96]:
for link in links:
    url = ("https://meduza.io" + str(link))
    ind_release = requests.get(url)
    ind_release = ind_release.text
  
    soup_release = bs4.BeautifulSoup(ind_release, "html.parser")
    for title in soup_release.findAll ('h1', class_ = "SimpleTitle-root"):
        titles.append(title.text) # получили заголовок статьи
        
    for date in soup_release.findAll ('time', class_ = "Timestamp-root"):
        dates.append(date.text) # получили дату публикации
        
    for text in soup_release.findAll ('div', class_ = "GeneralMaterial-article"):
        t = ""
        for p in text.findAll('p'): #текст разбит на параграфы. Собираем их вместе
            t += p.text # каждый кусок текста добавляем в один общий
        texts.append(t) # весь текст записываем в список
        
    


In [97]:
texts

['Корреспондент телеканала CNN Джим Акоста лишился аккредитации в\xa0Белом доме. Это произошло вскоре после пресс-конференции в\xa0Вашингтоне, на\xa0которой президент Трамп вступил в\xa0перепалку с\xa0журналистом.Акоста опубликовал в\xa0твиттере видео, на\xa0котором его пропуск в\xa0Белый дом конфискует сотрудник Секретной службы США.Руководство телеканала осудило действия администрации Трампа, напрямую связав их\xa0с\xa0неудобными вопросами, которые Акоста задавал президенту на\xa0пресс-конференции.Поддержку журналисту также выразила ассоциация корреспондентов президентского пула США, потребовав пересмотреть решение о\xa0лишении аккредитации Акосты.7\xa0ноября на\xa0пресс-конференции по\xa0итогам промежуточных выборов в\xa0США Дональд Трамп раскритиковал журналиста CNN Джима Акосту, назвав его «грубым и\xa0ужасным человеком». Акоста несколько раз пытался задать президенту один и\xa0тот\xa0же вопрос, не\xa0отдавая микрофон девушке, помогавшей проводить пресс-конференцию. Пресс-секретар

In [92]:
dates

['05:53, 8 ноября 2018',
 '07:32, 8 ноября 2018',
 '07:29, 8 ноября 2018',
 '06:30, 8 ноября 2018',
 '06:55, 8 ноября 2018',
 '21:20, 7 ноября 2018',
 '19:28, 7 ноября 2018',
 '05:13, 8 ноября 2018',
 '05:47, 8 ноября 2018',
 '19:52, 7 ноября 2018',
 '19:51, 7 ноября 2018',
 '15:31, 7 ноября 2018',
 '18:38, 7 ноября 2018',
 '17:29, 7 ноября 2018',
 '14:01, 7 ноября 2018',
 '15:15, 7 ноября 2018']

Осталось все полученные данные поместить в DataFrame и сохранить

In [98]:
df = pd.DataFrame({'title': titles, 'date': dates, 'text': texts})
df.to_csv("Medusa_News.csv")
df.head()

,title,date,text
0,Журналиста CNN лишили аккредитации в Белом дом...,"05:53, 8 ноября 2018",Корреспондент телеканала CNN Джим Акоста лишил...
1,В Бурятии МВД потребовало от издания предостав...,"07:32, 8 ноября 2018",Республиканское МВД потребовало от бурятского ...
2,США обвинили Иран в попытке спрятать нефтяные ...,"07:29, 8 ноября 2018",Около десятка нефтяных танкеров Ирана отключил...
3,13 заключенных погибли во время бунта в колони...,"06:30, 8 ноября 2018",По меньшей мере 13 человек погибли во время шт...
4,В Москве задержали подозреваемого в поджоге храма,"06:55, 8 ноября 2018",Московская полиция задержала подозреваемого в ...


# VK API

Рассмотрим работу с API, на примере VK API. API ‒ программный интерфейс приложения, сокращение от Application Programming Interface. Этот интерфейс позволяет выполнять различные операции автоматически, через приложение. Если API нам нужен исключительно как источник данных, можно писать запросы, позволяющие обратиться к хранилищу информации внутри API. Если мы хотим управлять приложением, которое будет выполнять какие-то действия, удаленно, можно написать код, который будет, например, автоматически отвечать на сообщения, когда мы не онлайн, лайкать новый пост друга через 30 секунд после его появления, пересылать на почту фотографии, которые выложили участники диалога и прочее.

Мы будем работать с API социальной сети ВКонтакте. Рассмотрим API как источник данных, позволяющий выгрузить все посты со стены пользователя или сообщества. Для работы нам понадобится библиотека `vk`, ее нужно установить через `pip install vk` (это также можно сделать через сам блокнот с использованием !)

In [42]:
!pip install vk

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Импортируем библиотеку:

In [52]:
import vk

Теперь нужно авторизоваться: создать приложение и получить токен доступа к нему.

## Для авторизации:
1. [Создать](https://vk.com/apps?act=manage) приложение ВКонтакте (пройти по ссылке). Дать название, выбрать Standalone-приложение.
2. Включить приложение, сделать публичным (Настройки - Состояние и выбрать в выпадающем меню Приложение включено и видно всем).
3. Авторизоваться: скопировать строку ниже в браузер, в `client id` вместо 1 выставить `id` своего приложения (первая строка в настроках ‒ ID приложения). Если не хочется ни в чем ограничивать свое приложение, можно оставить `scope=all` (у приложения будет доступ ко всему, к чему есть доступ у пользователя).

>`https://oauth.vk.com/authorize?client_id=1&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token`

4. Скопировать `access token` из обновленной адресной строки (все после `access_token=` и до `&expires_in`, без &). Никому не показывать! По этому токену можно получить доступ ко всему аккаунту.

In [53]:
# скопировать свой access token без пробелов вместо 1234
token = 1234
session = vk.Session(access_token = token) # открыть сессию для работы
api = vk.API(session) # подключиться к API

Попробуем сгрузить первые 100 постов со [стены](https://vk.com/hseofficial) неофициального сообщества ВШЭ. Сохраним ссылку на сообщество в переменную `group`:

In [54]:
group = 'hseofficial'

Обратите внимание: ссылка должна быть относительной, то есть без части с `https:/vk.com/`. Python и так будет знать, что мы работаем с сетью ВКонтакте.

Теперь получим доступ к стене этого сообщества:

In [55]:
# wall.get - "подключаемся" к стене
# count - сколько постов выгрузить (максимум)
# v - версия API, можно обойтись без нее, но Python может выдать warning

res = api.wall.get(domain = group, count = 100, v = 5.73)

Сохраним теперь полученные данные в виде JSON файла. Для этого импортируем модуль json. Откроем файл для записи и запишем туда все наши полученные данные.

In [56]:
import json
with open('vk.json', 'w') as f:
  json.dump(res, f, ensure_ascii=False)

# VK JSON Parser

Теперь научимся еще и загружать JSON файл с диска. Для этого нам попрежнему понадобится модуль json.

In [103]:
import json
with open('vk.json',  'r', encoding='Utf-8') as json_data:
    res_loaded = json.load(json_data)

In [115]:
res_loaded['items'][:2]

[{'id': 32494,
  'from_id': -132,
  'owner_id': -132,
  'date': 1541422799,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': '',
  'attachments': [{'type': 'link',
    'link': {'url': 'http://family.hse.ru/event/view/2271',
     'title': 'День карьеры факультета бизнеса и менеджмента',
     'caption': 'family.hse.ru',
     'description': '',
     'photo': {'id': 456239093,
      'album_id': -2,
      'owner_id': 100,
      'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
      'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
      'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
      'width': 150,
      'height': 80,
      'text': '',
      'date': 1541422799}}}],
  'post_source': {'type': 'vk'},
  'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
  'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
  'reposts': {'count': 2, 'user_reposted': 0},
  'vi

Результат, сохраненный в `res`, очень похож на словарь. На самом деле, многие API возвращают результаты в формате JSON (JavaScript Object Notation), который тоже может быть представлен в виде набора пар ключ-значение.

In [116]:
res_loaded.keys()

dict_keys(['count', 'items'])

Ключами являются `count` и `items`. Нужные нам объекты (текст постов, id автора, дата и время публикации и проч.) находятся в `items`.

In [62]:
res_loaded['items'][0] # первый элемент items - первый пост со всей информацией о нем

{'id': 32494,
 'from_id': -132,
 'owner_id': -132,
 'date': 1541422799,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'attachments': [{'type': 'link',
   'link': {'url': 'http://family.hse.ru/event/view/2271',
    'title': 'День карьеры факультета бизнеса и менеджмента',
    'caption': 'family.hse.ru',
    'description': '',
    'photo': {'id': 456239093,
     'album_id': -2,
     'owner_id': 100,
     'photo_75': 'https://pp.userapi.com/c850336/v850336548/628ea/TwPjzdADVZ4.jpg',
     'photo_130': 'https://pp.userapi.com/c850336/v850336548/628eb/gMSUSljxiC8.jpg',
     'photo_604': 'https://pp.userapi.com/c850336/v850336548/628ec/kIMEO_f_3JQ.jpg',
     'width': 150,
     'height': 80,
     'text': '',
     'date': 1541422799}}}],
 'post_source': {'type': 'vk'},
 'comments': {'count': 0, 'can_post': 1, 'groups_can_post': True},
 'likes': {'count': 6, 'user_likes': 0, 'can_like': 1, 'can_publish': 1},
 'reposts': {'count': 2, 'user_reposted': 0},
 'views': {'count': 2832}}

Помимо текста поста можно найти много всего интересного. Например, тип поста (`post_type`), дата (`date`), id поста (`id`), лайки (`likes`, которые включают информацию о том, могут ли пользователи лайкать пост и публиковать его, а также собственно число лайков), репосты (`reposts`, которые включают число репостов), число просмотров (`views`), комментарии (`comments`, которые включают информацию о том, могут ли пользователи комментировать пост, и число комментариев), и так далее.

Давайте остановимся на тексте поста, id автора, id поста и дате публикации. Чтобы извлечь соответствующую информацию, сохраним `items` и извлечем из них нужные поля:

In [117]:
items = res_loaded['items']
full_list = []

for item in items:
    l = [item['from_id'], item['id'], item['text'], item['date']] # нужные поля
    full_list.append(l) # добавляем в список списков full_list
# несколько элементов списка
full_list[0:4]

[[-132, 32494, '', 1541422799],
 [-132,
  32493,
  'На Шаболовке 7 ноября в 18:30 ауд 5215 состоится встреча с представителем оргкомитета симпозиума, Lars John. Авторы лучших эссе получат возможность участвовать в симпозиуме в Швейцарии, а автор cамого лучшего эссе получит значительный денежный приз. Приглашаем всех студентов магистратуры, студентов 4 курса и аспирантов принять участие в этой встрече. \n \nTo all grad/postgrad and 4th year undergraduate students: compete until 1 Feb 2019 and qualify as a Leader of Tomorrow for the #49sgs (8–10 May 2019) in Switzerland, all expenses paid. Create an impact and win CHF 20,000. Leaders who have attended in the past include Niall Ferguson, Jack Ma, Christine Lagarde, Paul Polman, Anders Fogh Rasmussen, Ratan Tata, Muhammad Yunus, and many more. Register now at www.symp.sg/apply',
  1541076555],
 [-132, 32492, '', 1540743239],
 [-132, 32491, '', 1540551399]]

Видно, что в двух первых постах текста не обнаружено, там только картинки, ссылки и репосты.

Из этого списка списков можно легко сделать датафрейм `pandas`. Но прежде посмотрим, как сгрузить следующие 100 (и не только 100) постов со стены. Обычно при работе с API нужно принимать во внимание две вещи: 
1. Какое ограничение стоит на число запросов за один раз (число постов в нашем случае) 
2. Какое ограничение стоит на число запросов в минуту. 

Чтобы действовать в соответствии с этими ограничениями, поступим так: будем грузить каждые следующие 100 постов, добавлять их к нашему списку, потом немного ждать, и грузить еще 100 постов, и так до тех пор, пока не сгрузим желаемое количество.

Давайте для начала выберем это желаемое число постов. Пусть будет 400.

In [64]:
nposts = 400

Теперь вопрос: по каким значениям нужно "пробегаться" в цикле, чтобы сгрузить посты с 100 по 400 (первые 100 уже сохранены в `res`)? По целым значениям от 2 до 4 включительно, умножая эти значения на 100. В `vk.get` есть опция `offset`. Она позволяет "сдвинуть" начало отсчета постов на некоторое число. Так, если мы напишем `api.wall.get(domain = group, count = 100, offset = 100, v = 5.73)`, мы получим посты с 100 по 200, так как начало отсчета сдвинулось на 100.

Реализуем описанное выше. Для цикла нам понадобится `range()`, а для задержки после выгрузки каждой сотни постов нам пригодится функция `sleep`:

In [65]:
from time import sleep

In [66]:
for i in range(2, int(nposts/100) + 1):
    res2 = api.wall.get(domain = group, count = 100, offset = 100 * i, v = 5.73)
    items2 = res2['items']
    items.extend(items2) # добавляем к первой сотне постов в items
    sleep(10)

Но если у ссылок, репостов и картинок нет текста, то наш DataFrame будет выглядет неполноценно. Поэтому добавим условие, что если текст отсутсвует, вставим в DataFrame строку с этой отметкой.

In [119]:
# опять выберем только нужные поля
full_list = []
for item in items:
    if item['text'] == "":
        l = [item['from_id'], item['id'], "Картинка, ссылка или репост", item['date']]
    else:
        l = [item['from_id'], item['id'], item['text'], item['date']]
    full_list.append(l)

Оставлось превратить обновленный список `items` (список списков) в датафрейм. Импортируем `pandas`.

In [68]:
import pandas as pd

Создадим датафрейм:

In [120]:
df = pd.DataFrame(full_list)
df.head(10)

,0,1,2,3
0,-132,32494,"Картинка, ссылка или репост",1541422799
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555
2,-132,32492,"Картинка, ссылка или репост",1540743239
3,-132,32491,"Картинка, ссылка или репост",1540551399
4,-132,32488,"Картинка, ссылка или репост",1540302306
5,-132,32486,26-28 октября в Москве пройдет крупнейший в Ро...,1539863523
6,-132,32485,"Картинка, ссылка или репост",1539781575
7,-132,32484,"Картинка, ссылка или репост",1539718883
8,-132,32481,"Картинка, ссылка или репост",1539088885
9,-132,32480,Будущим магистрам! Новая программа Факультета ...,1539088745


Ура! Осталось только дать внятные названия столбцам и разобраться, почему дата представлена в таком виде. что делать со столбцами, мы уже знаем.

In [121]:
df.columns = ['From_id', 'Id', 'Text', 'Date_Unix']
df.head(10)

,From_id,Id,Text,Date_Unix
0,-132,32494,"Картинка, ссылка или репост",1541422799
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555
2,-132,32492,"Картинка, ссылка или репост",1540743239
3,-132,32491,"Картинка, ссылка или репост",1540551399
4,-132,32488,"Картинка, ссылка или репост",1540302306
5,-132,32486,26-28 октября в Москве пройдет крупнейший в Ро...,1539863523
6,-132,32485,"Картинка, ссылка или репост",1539781575
7,-132,32484,"Картинка, ссылка или репост",1539718883
8,-132,32481,"Картинка, ссылка или репост",1539088885
9,-132,32480,Будущим магистрам! Новая программа Факультета ...,1539088745


С датой все интереснее. То, что указано в столбце `date`, это дата в виде UNIX-времени (POSIX-времени). Это число секунд, прошедших с 1 января 1970 года. Несмотря на то, что такой формат даты-времени кажется необычным, он довольно широко распространен в разных системах и приложениях. Этот факт, конечно, радует, но хочется получить дату в более человеческом формате. Давайте напишем функцию для перевода UNIX-времени в формат год-месяц-день-часы-минуты-секунды. Для этого нам понадобится модуль datetime.

In [122]:
from datetime import datetime

In [124]:
def date_norm(date):
    d = datetime.fromtimestamp(date) # timestamp - UNIX-время в виде строки
    str_d = d.strftime("%Y-%m-%d %H:%M:%S") # %Y-%m-%d %H:%M:%S - год-месяц-день, часы:минуты:секунды
    date_norm, time_norm = str_d.split(' ') # разобьем результат на части, отделим дату от времени
    return date_norm, time_norm

Применим нашу функцию к элементам столбца date и создадим новый ‒ `date_norm`.

In [125]:
df['Date_Norm'] = df.Date_Unix.apply(date_norm)

In [126]:
df.head()

,From_id,Id,Text,Date_Unix,Date_Norm
0,-132,32494,"Картинка, ссылка или репост",1541422799,"(2018-11-05, 15:59:59)"
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,"(2018-11-01, 15:49:15)"
2,-132,32492,"Картинка, ссылка или репост",1540743239,"(2018-10-28, 19:13:59)"
3,-132,32491,"Картинка, ссылка или репост",1540551399,"(2018-10-26, 13:56:39)"
4,-132,32488,"Картинка, ссылка или репост",1540302306,"(2018-10-23, 16:45:06)"


Можно было, конечно, не разбивать на части дату и время, сохранять одной строкой. А можно написать функции, которые будут отделять дату от времени ‒ извлекать их из кортежа в date_norm.

In [127]:
def get_date(date):
    return date[0]

def get_time(date):
    return date[1]

In [128]:
df['Date'] = df.Date_Norm.apply(get_date)
df['Time'] = df.Date_Norm.apply(get_time)
df.head()

,From_id,Id,Text,Date_Unix,Date_Norm,Date,Time
0,-132,32494,"Картинка, ссылка или репост",1541422799,"(2018-11-05, 15:59:59)",2018-11-05,15:59:59
1,-132,32493,На Шаболовке 7 ноября в 18:30 ауд 5215 состоит...,1541076555,"(2018-11-01, 15:49:15)",2018-11-01,15:49:15
2,-132,32492,"Картинка, ссылка или репост",1540743239,"(2018-10-28, 19:13:59)",2018-10-28,19:13:59
3,-132,32491,"Картинка, ссылка или репост",1540551399,"(2018-10-26, 13:56:39)",2018-10-26,13:56:39
4,-132,32488,"Картинка, ссылка или репост",1540302306,"(2018-10-23, 16:45:06)",2018-10-23,16:45:06


Всё! Материалы о разных методах и функциях для `vk.api` можно найти в [официальной документации](https://vk.com/dev/manuals).